## Agentic Retrieval in Azure AI Search and Azure AI Agent Service


### 1. Load Connections

In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
project_conn_str = os.environ["PROJECT_CONNECTION_STRING"]
agent_model = os.getenv("AGENT_MODEL", "gpt-4o")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4o")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4o")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")
api_version = "2025-05-01-Preview"

### 2. Create Search Index

In [2]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")


Index 'earth_at_night' created or updated successfully


### 3. Upload sample documents

In [3]:
import requests
from azure.search.documents import SearchIndexingBufferedSender
from azure.search.documents import SearchClient

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}'")


Documents uploaded to index 'earth_at_night'


### 4. Create search agent

In [4]:

from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex, KnowledgeAgentRequestLimits, AzureOpenAIVectorizerParameters
from azure.search.documents.indexes import SearchIndexClient

agent = KnowledgeAgent(
    name=agent_name,
    models=[
        KnowledgeAgentAzureOpenAIModel(
            azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,
                deployment_name=azure_openai_gpt_deployment,
                model_name=azure_openai_gpt_model
            )
        )
    ],
    target_indexes=[
        KnowledgeAgentTargetIndex(
            index_name=index_name,
            default_reranker_threshold=2.5
        )
    ],
    request_limits=KnowledgeAgentRequestLimits(
        max_output_size=10000
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_agent(agent)
print(f"Knowledge agent '{agent_name}' created or updated successfully")

Knowledge agent 'earth-search-agent' created or updated successfully


### 5. Create AI Agent

In [ ]:
from azure.ai.projects import AIProjectClient
import json
import logging
logging.basicConfig(level=logging.DEBUG)

def on_request(pipeline):
    print(pipeline.http_request.url, pipeline.http_request.body)

project_client = AIProjectClient.from_connection_string(project_conn_str, credential=credential, raw_request_hook=on_request)

instructions = """
An Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""
agent = project_client.agents.create_agent(
    model=agent_model,
    name=agent_name,
    instructions=instructions
)
print(json.dumps(agent.as_dict(), indent=2))

AttributeError: 'PipelineRequest' object has no attribute 'url'

### 6. Add Agentic Retrieval tool to AI Agent

In [6]:
from azure.ai.projects.models import FunctionTool, ToolSet, ListSortOrder

from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams

agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, credential=credential)

thread = project_client.agents.create_thread()
retrieval_results = {}

def agentic_retrieval() -> str:
    """
        Searches a NASA e-book about images of Earth at night and other science related facts.
        The returned string is in a JSON format that contains the reference id.
        Be sure to use the same format in your agent's response
    """
    # Take the last 5 messages in the conversation
    messages = project_client.agents.list_messages(thread.id, limit=5, order=ListSortOrder.DESCENDING)
    # Reverse the order so the most recent message is last
    messages.data.reverse()
    retrieval_result = agent_client.knowledge_retrieval.retrieve(
        retrieval_request=KnowledgeAgentRetrievalRequest(
            messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg.content[0].text)]) for msg in messages.data],
            target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
        )
    )

    # Associate the retrieval results with the last message in the conversation
    last_message = messages.data[-1]
    retrieval_results[last_message.id] = retrieval_result

    # Return the grounding response to the agent
    return retrieval_result.response[0].content[0].text

# Link to AI Agent Service function calling tutorial
user_functions = { agentic_retrieval }
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
project_client.agents.enable_auto_function_calls(toolset=toolset)


### 7. Start a chat with the agent

In [7]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
)

run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.list_messages(thread_id=thread.id).get_last_text_message_by_role("assistant").text.value

print("Agent response:", output.replace(".", "\n"))


Agent response: Suburban belts display larger December brightening than urban cores despite higher absolute light levels downtown due to the increased spread of residential holiday lighting
 People in suburban areas often decorate their homes and neighborhoods more extensively with holiday lights, which adds to the overall brightness during the holiday season
 In contrast, urban cores, primarily composed of commercial buildings and less residential decoration, do not experience such a significant increase in lighting (ref_id: 1)


The Phoenix nighttime street grid is sharply visible from space because of its well-organized grid layout and the extensive lighting infrastructure within the city and its suburbs
 Unlike the vast stretches of interstates between midwestern cities, which remain dim due to the sparsely populated areas and the focus on long-distance travel rather than local street lighting, the Phoenix metropolitan area features a dense network of illuminated streets, commercia

### 7.1: Review retrieval activity and results

In [8]:

retrieval_result = retrieval_results.get(message.id)
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))


Retrieval activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1359,
    "output_tokens": 440
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "suburban belts December brightening urban cores light levels"
    },
    "query_time": "2025-05-06T16:02:09.628Z",
    "elapsed_ms": 372
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "Phoenix nighttime street grid visibility from space"
    },
    "query_time": "2025-05-06T16:02:10.034Z",
    "count": 2,
    "elapsed_ms": 405
  },
  {
    "id": 3,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "midwestern cities interstate dim visibility from space"
    },
    "query_time": "2025-05-06T16:02:10.479Z",
    "count": 1,
    "elapsed_ms": 444
  }
]
Retrieval results
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
  

### 8. Continue the conversation

In [9]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="How do I find lava at night?"
)

run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")

output = project_client.agents.list_messages(thread_id=thread.id).get_last_text_message_by_role("assistant").text.value
print("Agent response:", output.replace(".", "\n"))


Agent response: To find lava at night, you can use various methods and tools that allow for the observation and monitoring of volcanic activity
 Here are some key points on how to locate and identify lava flows at night:

1
 **Satellite Imagery and Thermal Imaging**:
   - Satellites equipped with thermal sensors, such as the VIIRS (Visible Infrared Imaging Radiometer Suite) Day/Night Band (DNB) on the Suomi NPP satellite, can capture nighttime images of lava flows

   - Thermal imaging is particularly useful for detecting the heat emitted from volcanic activity, allowing for the identification of hot lava even in the dark

   - For example, the VIIRS DNB captured nighttime images of lava flows on Mount Etna in Sicily, Italy, showcasing bright spots indicating active volcanic regions (ref_id: 1)


2
 **Monitoring Active Volcanoes**:
   - Regular monitoring of known active volcanoes, such as Mount Etna or Kilauea, can help in identifying and tracking lava flows
 During eruptions, these v

### 8.1: Review retrieval activity and results

In [10]:
retrieval_result = retrieval_results.get(message.id)
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))

Retrieval activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1651,
    "output_tokens": 313
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "how to locate lava flows at night"
    },
    "query_time": "2025-05-06T16:02:27.904Z",
    "count": 6,
    "elapsed_ms": 509
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "best practices for observing lava at night"
    },
    "query_time": "2025-05-06T16:02:28.370Z",
    "elapsed_ms": 466
  },
  {
    "id": 3,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "safety tips for finding lava at night"
    },
    "query_time": "2025-05-06T16:02:28.852Z",
    "elapsed_ms": 482
  }
]
Retrieval results
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
    "activity_source": 1,
    "doc_key": "earth_at_night_508_page_60_ve